In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import re
import warnings
import html
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
!unzip '/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/google-quest-challenge.zip'
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_train.shape, df_test.shape

Archive:  /content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/google-quest-challenge.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


((6079, 41), (476, 11))

In [5]:
# 1. setting up target features
question_tar = ['question_asker_intent_understanding',
       'question_body_critical', 'question_conversational',
       'question_expect_short_answer', 'question_fact_seeking',
       'question_has_commonly_accepted_answer',
       'question_interestingness_others', 'question_interestingness_self',
       'question_multi_intent', 'question_not_really_a_question',
       'question_opinion_seeking', 'question_type_choice',
       'question_type_compare', 'question_type_consequence',
       'question_type_definition', 'question_type_entity',
       'question_type_instructions', 'question_type_procedure',
       'question_type_reason_explanation', 'question_type_spelling',
       'question_well_written']
       
answer_tar = ['answer_helpful',
       'answer_level_of_information', 'answer_plausible', 'answer_relevance',
       'answer_satisfaction', 'answer_type_instructions',
       'answer_type_procedure', 'answer_type_reason_explanation',
       'answer_well_written']
 
tar_features = question_tar + answer_tar
len(tar_features)

30

# 4.1 BERT

In [6]:
!pip install transformers

     |████████████████████████████████| 778kB 8.7MB/s 
     |████████████████████████████████| 890kB 49.3MB/s 
     |████████████████████████████████| 3.0MB 50.2MB/s 
     |████████████████████████████████| 1.1MB 55.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=88ee00994acbababe20e39c3c2e63e6656373bee44b7b58510bff6f3e7308284
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [7]:
# 1. defining model and tokenizer
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, BertConfig
 
# Load pretrained model/tokenizer
config = BertConfig.from_pretrained('bert-base-uncased', output_hidden_states=True)
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', config = config)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
# 2. tokenizer
title_body_train_tokens = bert_tokenizer.batch_encode_plus(title_body_train, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
ans_train_tokens = bert_tokenizer.batch_encode_plus(answer_train, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")

In [ ]:
# 3. bert model prediction
x = bert_model([ans_train_input_ids[i*batch_size:(i+1)*batch_size], ans_train_attn_mask[i*batch_size:(i+1)*batch_size], ans_train_token_typ_ids[i*batch_size:(i+1)*batch_size]])[2]
lst = [i[:, 0, :] for i in x[-4:]]

# 4.2. USE Embeddings

In [8]:
import tensorflow as tf
import tensorflow_hub as hub

In [9]:
# loading USE model
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

INFO:absl:Using /tmp/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder-large/5, Total size: 577.10MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder-large/5'.


# 4.3. Sentence Embedding : RoBERTa

In [10]:
import tensorflow as tf
from transformers import RobertaConfig, RobertaTokenizer, TFRobertaModel
 
# Load pretrained model/tokenizer
config = RobertaConfig.from_pretrained('roberta-base', output_hidden_states=True)
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base', config = config)

- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFRobertaModel for predictions without further training.


# 4.4. Sentence Embedding : XLNet

In [11]:
import tensorflow as tf
from transformers import XLNetConfig, XLNetTokenizer, TFXLNetModel
 
# Load pretrained model/tokenizer
config = XLNetConfig.from_pretrained('xlnet-base-cased', output_hidden_states=True)
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
xlnet_model = TFXLNetModel.from_pretrained('xlnet-base-cased', config = config)

- This IS expected if you are initializing TFXLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFXLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFXLNetModel for predictions without further training.


# 5. Model

In [12]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, BatchNormalization
from tensorflow.keras.models import Model

In [13]:
class block(tf.keras.layers.Layer):
  def __init__(self, d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.1, seed = 42):
    super().__init__()
    self.d1_rate = d1_rate
    self.dense_1_units = dense_1_units
    self.d2_rate = d2_rate
    self.dense_2_units = dense_2_units
    self.d3_rate = d3_rate
    self.seed = seed

  def build(self, input_shape):
    # self.inp = Input(name = 'ans_bert_out', shape = (768*4,), dtype = 'float32')
    self.dropout_1 =  Dropout(rate =self.d1_rate, seed = self.seed)

    self.dense_1 = Dense(units = self.dense_1_units, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = self.seed))
    self.dropout_2 = Dropout(rate = self.d2_rate, seed = self.seed)

    self.dense_2 = Dense(units = self.dense_2_units, activation = 'relu', kernel_initializer = tf.keras.initializers.he_normal(seed = self.seed))
    self.dropout_3 = Dropout(rate = self.d3_rate, seed = self.seed)

  def call(self, X):
    dropout_1 = self.dropout_1(X)

    dense_1 = self.dense_1(dropout_1)
    dropout_2 = self.dropout_2(dense_1)

    dense_2 = self.dense_2(dropout_2)
    dropout_3 = self.dropout_3(dense_2)
    return dropout_3

In [15]:
tf.keras.backend.clear_session()
seed = 42

# *-----------------title_body-----------------*
tb_bert_input = Input(name = 'tb_bert_out', shape = (768*4,), dtype = 'float32')
tb_bert_block = block(d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.1, seed = 42)(tb_bert_input)
tb_bert_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_bert_block)

tb_use_input = Input(name = 'tb_use_out', shape = (512,), dtype = 'float32')
tb_use_block = block(d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.2, seed = 42)(tb_use_input)
tb_use_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_use_block)

tb_roberta_input = Input(name = 'tb_roberta_out', shape = (768*4,), dtype = 'float32')
tb_roberta_block = block(d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.1, seed = 42)(tb_roberta_input)
tb_roberta_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_roberta_block)

tb_xlnet_input = Input(name = 'tb_xlnet_out', shape = (768*4,), dtype = 'float32')
tb_xlnet_block = block(d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.1, seed = 42)(tb_xlnet_input)
tb_xlnet_out = Dense(units = 21, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(tb_xlnet_block)

tb_out = tf.reduce_mean([tb_bert_out ,tb_use_out ,tb_roberta_out ,tb_xlnet_out], axis = 0)

# *----------------- answer -----------------*
ans_bert_input = Input(name = 'ans_bert_out', shape = (768*4,), dtype = 'float32')
ans_bert_block = block(d1_rate = 0.2, dense_1_units = 2048, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.1, seed = 42)(ans_bert_input)
ans_bert_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_bert_block)

ans_use_input = Input(name = 'ans_use_out', shape = (512,), dtype = 'float32')
ans_use_concat = tf.concat([tb_use_out, ans_use_input], axis = -1) 
ans_use_block = block(d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.2, seed = 42)(ans_use_concat)
ans_use_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_use_block)

ans_roberta_input = Input(name = 'ans_roberta_out', shape = (768*4,), dtype = 'float32')
ans_roberta_concat = tf.concat([tb_roberta_out, ans_roberta_input], axis = -1) 
ans_roberta_block = block(d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.1, seed = 42)(ans_roberta_concat)
ans_roberta_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_roberta_block)

ans_xlnet_input = Input(name = 'ans_xlnet_out', shape = (768*4,), dtype = 'float32')
ans_xlnet_block = block(d1_rate = 0.2, dense_1_units = 1024, d2_rate = 0.2, dense_2_units = 512, d3_rate = 0.1,  seed = 42)(ans_xlnet_input)
ans_xlnet_out = Dense(units = 9, activation = 'sigmoid', kernel_initializer = tf.keras.initializers.he_normal(seed = seed))(ans_xlnet_block)

ans_out = tf.reduce_mean([ans_bert_out ,ans_use_out ,ans_roberta_out ,ans_xlnet_out], axis = 0)

# *----------------- concat -----------------*
out =  Concatenate(axis = -1)([tb_out, ans_out])

model = Model(inputs = [tb_bert_input, tb_use_input, tb_roberta_input, tb_xlnet_input, ans_bert_input, ans_use_input, ans_roberta_input, ans_xlnet_input], outputs = out)
model.load_weights('/content/drive/My Drive/AAIC Course/Personal case study 2  - Google Quest/weights.-029-0.37055')
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tb_use_out (InputLayer)         [(None, 512)]        0                                            
__________________________________________________________________________________________________
tb_roberta_out (InputLayer)     [(None, 3072)]       0                                            
__________________________________________________________________________________________________
block_1 (block)                 (None, 512)          1050112     tb_use_out[0][0]                 
__________________________________________________________________________________________________
block_2 (block)                 (None, 512)          3671552     tb_roberta_out[0][0]             
_______________________________________________________________________________________

In [16]:
# post processing function : binning
def return_bins(arr):
  val = np.unique(arr)
  bins = []
  for i in range(len(val)):
    if i > 0:
      bins.append((val[i-1] + val[i])/2)
  return bins

unique_val_30 = [np.unique(df_train[tar_features].values[:, i]) for i in range(30)]
bins_30 = [return_bins(df_train[tar_features].values[:, i]) for i in range(30)]

def binned_out(y_pred):
  col = y_pred.shape[1]
  final_pred = np.zeros(y_pred.shape)
  for i in range(col):
    idx = np.digitize(y_pred[:, i], bins_30[i])
    final_pred[:, i] = unique_val_30[i][idx]
  return final_pred

# 6. Final_pipeline:

In [59]:
# defining a function to remove stop_words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.remove('no'); stop_words.remove('not'); stop_words.remove('nor')
 
def stopwrd_removal(sent):
  lst = []
  for wrd in sent.split():
    if wrd not in stop_words:
      lst.append(wrd)
  return " ".join(lst)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
# 1. input text preprocessor
def text_preprocessor(corpus, stop_word = False, remove_specialchar = False):
  clean_corpus = []
  for doc in corpus:
    # 1. remove html tags, html urls, replace html comparison operators
    clean_str = re.sub('<.*?>', '', doc)
    clean_str = clean_str.replace('&lt;', '<')\
                .replace('&gt;', '>')\
                .replace('&le;', '<=' )\
                .replace('&ge;', '>=')

    # 2. remove latex i,e., mostly formulas since it's mathematics based dataset
    clean_str = re.sub('\$.*?\$', '', clean_str)

    # 3. all lowercase 
    clean_str = clean_str.lower()

    # 4. decontractions
    clean_str = clean_str.replace("won't", "will not").replace("can\'t", "can not").replace("n\'t", " not").replace("\'re", " are").\
                                                  replace("\'s", " is").replace("\'d", " would").replace("\'ll", " will").\
                                                  replace("\'t", " not").replace("\'ve", " have").replace("\'m", " am")

    # 5. removing non-english or hebrew characters
    clean_str = clean_str.encode("ascii", "ignore").decode()

    # 6. remove all special-characters other than alpha-numericals
    if remove_specialchar == True:
      clean_str = re.sub('[^A-Za-z0-9]+', ' ', clean_str)

    # 7. Stop_word removal
    if stop_word == True:
      clean_str = stopwrd_removal(clean_str)

    # 8. remove all white-space i.e., \n, \t, and extra_spaces
    clean_str = re.sub('  +', ' ', clean_str)
    clean_str = clean_str.replace("\n", " ").replace("\t", " ").strip()

    clean_corpus.append(clean_str)
  return clean_corpus

In [51]:
# 2. get all sentence embeddings 
def get_embeddings(title_body_pr, answer_pr):
  # 2.1 BERT_embeddings
  title_body_BERT_tokens = bert_tokenizer.batch_encode_plus(title_body_pr, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf") # Tokenizing
  ans_BERT_tokens = bert_tokenizer.batch_encode_plus(answer_pr, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
  x_BERT = bert_model([title_body_BERT_tokens['input_ids'], title_body_BERT_tokens['attention_mask'], title_body_BERT_tokens['token_type_ids']])[2] # Prediction
  tb_BERT = tf.concat([tf.reduce_mean(i, axis = 1) for i in x_BERT[-4:]], axis = 1)
  x_BERT = bert_model([ans_BERT_tokens['input_ids'], ans_BERT_tokens['attention_mask'], ans_BERT_tokens['token_type_ids']])[2]
  ans_BERT = tf.concat([i[:, 0, :] for i in x_BERT[-4:]], axis = 1)

  # 2.2 USE_embeddings
  tb_USE = use_model(title_body_pr)
  ans_USE = use_model(answer_pr)

  # 2.3 RoBERTa_embeddings
  title_body_RoBERTa_tokens = roberta_tokenizer.batch_encode_plus(title_body_pr, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf") # Tokenizing
  ans_RoBERTa_tokens = roberta_tokenizer.batch_encode_plus(answer_pr, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
  x_RoBERTa = roberta_model([title_body_RoBERTa_tokens['input_ids'], title_body_RoBERTa_tokens['attention_mask']])[2] # Prediction
  tb_RoBERTa = tf.concat([i[:, 0, :] for i in x_RoBERTa[-4:]], axis = 1)
  x_RoBERTa = roberta_model([ans_RoBERTa_tokens['input_ids'], ans_RoBERTa_tokens['attention_mask']])[2]
  ans_RoBERTa = tf.concat([i[:, 0, :] for i in x_RoBERTa[-4:]], axis = 1)

  # 2.4 XLNet_embeddings
  title_body_XLNet_tokens = xlnet_tokenizer.batch_encode_plus(title_body_pr, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf") # Tokenizing
  ans_XLNet_tokens = xlnet_tokenizer.batch_encode_plus(answer_pr, max_length = 512, truncation = True, pad_to_max_length = True, return_tensors="tf")
  x_XLNet = xlnet_model([title_body_XLNet_tokens['input_ids'], title_body_XLNet_tokens['attention_mask']])[1] # Prediction
  tb_XLNet = tf.concat([tf.reduce_mean(i, axis = 1) for i in x_XLNet[-4:]], axis = 1)
  x_XLNet = xlnet_model([ans_XLNet_tokens['input_ids'], ans_XLNet_tokens['attention_mask']])[1]
  ans_XLNet = tf.concat([tf.reduce_mean(i, axis = 1) for i in x_XLNet[-4:]], axis = 1)

  return [tb_BERT, tb_USE, tb_RoBERTa, tb_XLNet, ans_BERT, ans_USE, ans_RoBERTa, ans_XLNet]

In [58]:
# 3. final_function
def final_pipeline(ques_title, ques_body, answer):
  # 1. text_preprocessing
  title_body_pr = text_preprocessor([ques_title + ' ' + ques_body])
  answer_pr = text_preprocessor([answer])

  # 2. get all sentence embeddings   
  ins = get_embeddings(title_body_pr, answer_pr)

  # 3. model_prediction
  pred = model.predict(x = ins)

  # 4. binned_output
  final_out = binned_out(pred)
  return final_out

### Final_prediction :

In [62]:
%%time
dp = 0
ques_title = df_train.question_title.values[dp]
ques_body = df_train.question_body.values[dp]
answer = df_train.answer.values[dp]

print(final_pipeline(ques_title, ques_body, answer))

[[0.88888889 0.55555556 0.         0.66666667 0.66666667 0.5
  0.66666667 0.5        0.5        0.         0.66666667 0.33333333
  0.         0.         0.         0.         0.5        0.
  0.5        0.         0.88888889 1.         0.66666667 1.
  1.         0.86666667 0.33333333 0.         0.66666667 0.88888889]]
CPU times: user 741 ms, sys: 26.4 ms, total: 767 ms
Wall time: 779 ms


# Summary :

In [ ]:
Please summarize your total work and the below topics.

1. The business problem you are solving. 
2. The loss function used and why
3. Metric used.
4. Final summary with findings and results.

### 1. Business Problem :

  * 'To build a predictive algorithm for different subjective aspects of question-answering'.
  * For any question-aswering platform, it is important to know that - how are the questions and its corresponding answers interacting with users with different contexts.
  * Provided title, body and answer pair we want to predict some 30 continuous target features, all are ranging from 0 to 1. 
    
### 2. Loss function :

The appropriate loss function for this particular problem is 'binary_cross_entropy'.
  
* $binary\_cross\_entropy(yi, pi) = -\sum_{i=1}^n{(y_i\log(p_i) + (1 - y_i)\log(1 - p_i))}$
    
Q. Why BCE? since its regression problem then why not MSE (mean_squared_error)?
   
- Answer is simple, all features are ranging from 0 to 1. if we take MSE of 0.51 and 0.65, $MSE =  (0.51-0.65)^2 = 0.0196$.
- MSE results are very smaller and hence gradients will also be very small.
- Ultimately the model will not able make any progress on training data.
- Inversely, BCE is made for values ranging between 0 to 1, and hence BCE is the appropriate loss function even if it's regression problem.

### 3. Business Metric :

- Appropriate metric for the current problem is **spearman__rank_correlation_coefficient (rho)**

* $\rho(X, Y) = \frac{\text{cov}(X_r,Y_r)}{\sigma_{xr} \sigma_{yr}}$

- Given two random variable (y_true) and (y_pred), computing how these arrays are correlated to each other. 
- **spearman__rank_correlation** is the robust metric which can capture - complex relationships between arrays upto moderate extent.
- Because, 'ranking' or 'the order of values' is the basis of computing covariance between variables, this enables 'spearman__rank_correlation' to capture relationships robustly.

### 4. Findings and summary :

1. RESULTS :

|         Model          | train_spearman | val_spearman |
|------------------------|----------------|--------------|
|     baseline_LSTM      |     0.4995     |   0.34899    |
|          BERT          |    0.50081     |   0.39536    |
|          USE           |    0.52245     |    0.4366    |
|        RoBERTa         |    0.48059     |   0.41772    |
|         XLNet          |    0.52237     |   0.37525    |
| BERT_USE_RoBERTa_XLNet |     0.5068     |   0.43924    |

  - Combination of 'BERT, USE, RoBERTa, XLNet' has given winning results but not with significant margin. Hence 'USE' is preferable.
  - So far, 'USE' embeddings are far more preferable at production environment for our task, Reasons are as follow :
    1. Comparitively, forward pass through 'USE' is not very expensive task in terms of time and computation.
    2. USE was actually built to capture semantic similarity.
    
2. No. of datapoints that actually matters in this kind of NLP problems. We dealt with very less no.of datapoints hence model failed to show further improvements.

3. Pretrained sentence embeddings are more powerful when :
    1. Manually trained on the similar data or the task as current problem has...
    2. Fine tuned on current dataset.
    
(NOTE : Both task are computationally very expensive, hence not able to introduced in the current work.)

4. Imbalanced data : in this experiment very basic and key learning is no model can deal with imbalanced data. Complex models like BERT can handle data imabalancing upto some extent if not completely. Hence better we get balanced and enough data.

5. 'Post processing : target feature binning', since the metric is 'spearman_rank_corr' which takes order into consideration. It became important to get similar values as (y_true) has in prediction results (y_pred). And that has shown smaller improvements if not very significant improvement.